# Reelgood Scraper

Workflow: 3  

Goal: Start with the list of reelgood movie urls and get the data for each movie. Split the urls into jobs. 

Result: The file ```reelgood_movie.csv``` is created.

In [1]:
import os
import time
import pandas as PD

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
# can't reload script in a different folder

import sys
sys.path.append('../')
import reelgood_scraper as RS 

## Create Scraping Jobs 

In [4]:
URL_FILE = '../data/reelgood_url.csv'
url_df = PD.read_csv(URL_FILE)
url_df.head()
url_df.info()

,title,year,reelgood_id
0,1,2013,1-2013
1,"10,000 BC",2008,10000-bc-2008
2,1000 Rupee Note,2016,1000-rupee-note-2016
3,1000 To 1,2014,1000-to-1-2014
4,$100 and a T-Shirt: A Documentary About Zines ...,2004,100-and-a-tshirt-a-documentary-about-zines-in-...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18496 entries, 0 to 18495
Data columns (total 3 columns):
title          18496 non-null object
year           18496 non-null int64
reelgood_id    18496 non-null object
dtypes: int64(1), object(2)
memory usage: 433.6+ KB


In [5]:
url_1_df = url_df[0:4499]
url_2_df = url_df[4500:8999]
url_3_df = url_df[9000:13499]
url_4_df = url_df[13500:]

url_1_df.shape
url_2_df.shape
url_3_df.shape
url_4_df.shape

(4499, 3)

(4499, 3)

(4499, 3)

(4996, 3)

## Scrape Rotation

In [6]:
scraper = RS.ReelgoodScraper()
scraper.log_in()

SyntaxError: invalid syntax (<ipython-input-6-f08dbef3aa6c>, line 1)

In [ ]:
t0 = time.time()
movie_ls = []

for idx, row in url_1_df.iterrows():
    if idx % 500 == 0:
        print(f'row: {idx}')
    rg_id = row['reelgood_id']
    movie_ls.append(scraper.get_movie_data(rg_id))

scraper.close()
t1 = time.time()
print(f'time: {(t1-t0)/60/60:.2f} hrs')

In [ ]:
movie_df = PD.DataFrame(movie_ls)
movie_df.head()
movie_df.info()

In [ ]:
save_path = r'../data/reelgood_mov_1.csv'
movie_df.to_csv(save_path, index=False)
f'{round(os.path.getsize(save_path) /1e6, 2)} mb'

## Merge Job Files

In [ ]:
st op = 
movie_1_df = PD.read_csv('../data/reelgood_mov_1.csv')
movie_2_df = PD.read_csv('../data/reelgood_mov_2.csv')
movie_3_df = PD.read_csv('../data/reelgood_mov_3.csv')
movie_4_df = PD.read_csv('../data/reelgood_mov_4.csv')

movie_1_df.shape
movie_2_df.shape
movie_3_df.shape
movie_4_df.shape

In [ ]:
movie_df = PD.concat([movie_1_df, movie_2_df, movie_3_df, movie_4_df])
movie_df.head()
movie_df.info()

In [ ]:
save_path = r'../data/reelgood_movie.csv'
movie_df.to_csv(save_path, index=False)
f'{round(os.path.getsize(save_path) /1e6, 2)} mb'